In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import h5py
import cv2
import os
import glob
import time

from keras.layers import merge
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adadelta, Adam, SGD
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

from collections import Counter

from keras.models import load_model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.models import Model
from keras.utils.np_utils import to_categorical
import shutil

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

import numpy as np
from various_utils import *

Using TensorFlow backend.
/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
def prep_test(te_path):
    t = time.time()
    X_test, test_ids = load_full(te_path)
    X_test2 = X_test.astype('float32')
    X_test2 = preprocess_input(X_test2)
    features = model.predict(X_test2)
    features = features.reshape(len(X_test2), -1)
    print('Reshaped: ', features.shape)
    print('Time it took to prepare test data:', time.time() - t)
    return X_test2, test_ids, features

def prep_train(tr_path):
    t = time.time()
    X_train, y_train, train_ids = load_train(tr_path)
    X_train2 = X_train.astype('float32')
    X_train2 = preprocess_input(X_train2)
    features = model.predict(X_train2, batch_size = 16)
    features = features.reshape(len(X_train2), -1)
    print('Reshaped: ', features.shape)
    print('Time it took to prepare test data:', time.time() - t)
    return X_train, y_train, train_ids, features


def fit_knn(neighs, features):
    knn = NearestNeighbors(n_neighbors = neighs, n_jobs = 6, metric = 'cosine', algorithm = 'brute')
    knn.fit(features)
    neighbors = knn.kneighbors(features)
    return neighbors

def normalize_probs(df):
    df2 = df.copy().iloc[:, :-1]
    df3 = df.copy().iloc[:, :-1]
    for i in df2.columns:
        df3[i] = df2[i] / df2.sum(axis = 1)
    return df3

def get_averaged(neighbors):
    neigh = fit_knn(neighbors, features_test)
    sub_knn = nn_probs(neigh)
    sub_knn = normalize_probs(sub_knn)
    sub_knn['image'] = sub1['image']
    return sub_knn


def get_similar(neighbors_list, ids, threshold, savename):
    imgs = []
    for i in range(len(neighbors_list[0])):
        for j in range(1, neighbors_list[0].shape[1]):
            if abs(neighbors_list[0][i][0] - neighbors_list[0][i][j]) <= threshold and abs(neighbors_list[0][i][0] - neighbors_list[0][i][j]) >= 0.0001:
                print('Image index:', neighbors_list[1][i][j])
                #print('Distances:', neighbors_list[0][i])
                #print('Neighbors:', neighbors_list[1][i])
                imgs.append(neighbors_list[1][i][j])
    imgs = list(set(imgs))
    imgs_todel = []
    for i in imgs:
        imgs_todel.append(ids[i])
        with open('{}.txt'.format(savename), 'a') as out:
            out.write(ids[i] + '\n')
    return imgs_todel

def delete_similar(src, imgstodel):
    folders = ['Type_1', 'Type_2', 'Type_3']
    dst = src + 'to_delete/'
    if 'to_delete/' not in os.listdir(src):
        os.mkdir(src + 'to_delete/')
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join(src, fld, '*.jpg')
        files = glob.glob(path)
        for todel in imgstodel:
            for fl in files:
                if todel in fl:
                    flbase = os.path.basename(fl)
                    shutil.move(fl, dst + flbase)
    return
               
def show_neighbors(X, neighbors, index, index2):
    print(neighbors[0][index])
    print(neighbors[1][index])
    fig, ax = plt.subplots(1,2, figsize = (20, 12))
    ax[0].imshow(X[index])
    ax[1].imshow(X[index2])
    return
    

full_path = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/full_data_renamed/'
base_model = VGG19(weights='imagenet', include_top = False, input_shape = (299, 299, 3))
model = Model(input=base_model.input, output=base_model.get_layer('block5_pool').output)

In [3]:
X_full = np.load('raw_combined_resized.npy')
features = np.load('raw_combined_features.npy')
ids = np.load('raw_combined_ids.npy')

In [4]:
n8 = fit_knn(8, features)

In [26]:
imgstodel_full = get_similar(n8, ids, 0.25, 'similar_combined_0.25')
delete_similar(full_path, imgstodel_full)

Image index: 644
Image index: 1215
Image index: 1099
Image index: 702
Image index: 155
Image index: 367
Image index: 223
Image index: 431
Image index: 1344
Image index: 151
Image index: 1135
Image index: 552
Image index: 543
Image index: 861
Image index: 555
Image index: 1430
Image index: 1030
Image index: 133
Image index: 976
Image index: 576
Image index: 1401
Image index: 70
Image index: 1395
Image index: 309
Image index: 934
Image index: 79
Image index: 1091
Image index: 676
Image index: 831
Image index: 66
Image index: 55
Image index: 859
Image index: 1288
Image index: 1324
Image index: 838
Image index: 1082
Image index: 483
Image index: 784
Image index: 8585
Image index: 37
Image index: 859
Image index: 66
Image index: 497
Image index: 686
Image index: 172
Image index: 398
Image index: 313
Image index: 185
Image index: 336
Image index: 156
Image index: 237
Image index: 1085
Image index: 8613
Image index: 210
Image index: 935
Image index: 37
Image index: 55
Image index: 859
Image i

Image index: 220
Image index: 885
Image index: 1333
Image index: 1311
Image index: 1421
Image index: 395
Image index: 537
Image index: 1387
Image index: 1223
Image index: 388
Image index: 1188
Image index: 226
Image index: 123
Image index: 1111
Image index: 1261
Image index: 1267
Image index: 1405
Image index: 428
Image index: 1184
Image index: 377
Image index: 387
Image index: 1098
Image index: 1125
Image index: 647
Image index: 8276
Image index: 573
Image index: 739
Image index: 276
Image index: 314
Image index: 1316
Image index: 1206
Image index: 1105
Image index: 1001
Image index: 1013
Image index: 637
Image index: 270
Image index: 1162
Image index: 748
Image index: 754
Image index: 635
Image index: 111
Image index: 704
Image index: 948
Image index: 1215
Image index: 1
Image index: 980
Image index: 623
Image index: 882
Image index: 8407
Image index: 312
Image index: 8380
Image index: 1160
Image index: 333
Image index: 152
Image index: 572
Image index: 988
Image index: 1294
Image in

Image index: 2744
Image index: 2755
Image index: 8571
Image index: 5060
Image index: 1856
Image index: 2046
Image index: 3486
Image index: 4999
Image index: 5624
Image index: 2362
Image index: 3592
Image index: 3645
Image index: 3028
Image index: 1686
Image index: 1970
Image index: 3766
Image index: 2186
Image index: 2686
Image index: 2784
Image index: 4771
Image index: 2474
Image index: 2166
Image index: 3921
Image index: 5011
Image index: 4551
Image index: 4036
Image index: 1732
Image index: 4734
Image index: 3018
Image index: 8514
Image index: 5136
Image index: 3292
Image index: 4624
Image index: 4803
Image index: 3400
Image index: 3817
Image index: 2430
Image index: 2546
Image index: 3157
Image index: 3838
Image index: 3830
Image index: 2849
Image index: 5204
Image index: 3865
Image index: 2255
Image index: 1629
Image index: 5581
Image index: 2036
Image index: 5581
Image index: 5204
Image index: 1624
Image index: 2849
Image index: 3830
Image index: 2255
Image index: 3865
Image inde

Image index: 4843
Image index: 4648
Image index: 2025
Image index: 3097
Image index: 5422
Image index: 4687
Image index: 4960
Image index: 5315
Image index: 1453
Image index: 5285
Image index: 4799
Image index: 5115
Image index: 2574
Image index: 3188
Image index: 4487
Image index: 2108
Image index: 2374
Image index: 8412
Image index: 2990
Image index: 2641
Image index: 2640
Image index: 2772
Image index: 2987
Image index: 3176
Image index: 4890
Image index: 3488
Image index: 3431
Image index: 4741
Image index: 5753
Image index: 2927
Image index: 3359
Image index: 2174
Image index: 4776
Image index: 5432
Image index: 2022
Image index: 3119
Image index: 1579
Image index: 2186
Image index: 2784
Image index: 4755
Image index: 1536
Image index: 4477
Image index: 8457
Image index: 2831
Image index: 2198
Image index: 1519
Image index: 4756
Image index: 3452
Image index: 5096
Image index: 1894
Image index: 4929
Image index: 5021
Image index: 2252
Image index: 2554
Image index: 1847
Image inde

Image index: 3442
Image index: 2856
Image index: 4043
Image index: 2995
Image index: 3420
Image index: 2526
Image index: 2593
Image index: 1919
Image index: 1721
Image index: 2112
Image index: 3098
Image index: 3721
Image index: 2309
Image index: 3220
Image index: 3327
Image index: 2340
Image index: 5280
Image index: 4412
Image index: 8217
Image index: 5334
Image index: 5151
Image index: 2040
Image index: 2086
Image index: 5330
Image index: 2370
Image index: 4243
Image index: 4036
Image index: 3317
Image index: 1931
Image index: 5757
Image index: 3598
Image index: 1537
Image index: 1918
Image index: 5132
Image index: 1992
Image index: 1949
Image index: 4647
Image index: 4433
Image index: 2069
Image index: 2457
Image index: 2352
Image index: 5565
Image index: 5335
Image index: 1844
Image index: 4208
Image index: 1923
Image index: 1923
Image index: 1844
Image index: 4207
Image index: 5335
Image index: 2008
Image index: 3553
Image index: 4530
Image index: 4547
Image index: 4322
Image inde

Image index: 1501
Image index: 1832
Image index: 3140
Image index: 4420
Image index: 2314
Image index: 8243
Image index: 5730
Image index: 1994
Image index: 5107
Image index: 4422
Image index: 3474
Image index: 2966
Image index: 3716
Image index: 3919
Image index: 4439
Image index: 2268
Image index: 1838
Image index: 2745
Image index: 8295
Image index: 4873
Image index: 1978
Image index: 4829
Image index: 1442
Image index: 1821
Image index: 2770
Image index: 1671
Image index: 5004
Image index: 5407
Image index: 4395
Image index: 5161
Image index: 3391
Image index: 4752
Image index: 3722
Image index: 2352
Image index: 4205
Image index: 3646
Image index: 3961
Image index: 1981
Image index: 1629
Image index: 5204
Image index: 1624
Image index: 2255
Image index: 2849
Image index: 3830
Image index: 3865
Image index: 2275
Image index: 2435
Image index: 1922
Image index: 8648
Image index: 2335
Image index: 4819
Image index: 4404
Image index: 4713
Image index: 8634
Image index: 2457
Image inde

Image index: 7429
Image index: 6863
Image index: 5817
Image index: 5896
Image index: 7639
Image index: 7025
Image index: 6183
Image index: 7220
Image index: 7353
Image index: 7435
Image index: 5894
Image index: 6258
Image index: 6412
Image index: 6800
Image index: 7614
Image index: 6176
Image index: 6562
Image index: 6626
Image index: 7140
Image index: 7708
Image index: 7421
Image index: 7377
Image index: 6977
Image index: 6785
Image index: 6595
Image index: 6377
Image index: 6430
Image index: 6464
Image index: 7151
Image index: 6597
Image index: 5989
Image index: 8707
Image index: 7408
Image index: 8226
Image index: 7497
Image index: 6385
Image index: 6103
Image index: 7477
Image index: 7025
Image index: 6389
Image index: 6891
Image index: 8230
Image index: 7405
Image index: 6847
Image index: 6285
Image index: 7996
Image index: 6259
Image index: 7548
Image index: 6050
Image index: 7289
Image index: 7837
Image index: 7800
Image index: 8133
Image index: 6657
Image index: 7002
Image inde

In [20]:
orig = len([x for x in imgstodel_full if 'original'  in x])
add = len([x for x in imgstodel_full if 'additional'  in x])
tests = len([x for x in imgstodel_full if 'test'  in x])

print(orig, add, tests)

238 2117 85
